# Data cleanup

Sometimes MATLAB writes individual entries into the localization files that are not easily parsed (or impossible to parse in ThunderSTORM). For this reason, I need to find a way to clean up the data files.

### Clean up of data

In [ ]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

In [ ]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV1_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

Unfortunately, the data in the uncertainty column was saved such that most of the numbers are floats, but some are strings representing floats and some are in a strange complex exponential form. Let's filter out these rows to make working with the data frame easier and to protoype a clean up routine.

First, I'll generate a mask to pick out the rows containing strings.

In [ ]:
stringMask = df['uncertainty [nm]'].map(lambda x: isinstance(x, str)).as_matrix()

Let's see what the strings look like:

In [ ]:
df['uncertainty [nm]'][stringMask]

So, strangely there were about 11,000 localizations that were interpreted as strings in this data set. Let's cast them as numeric data types. Some of the strings cannot be recognized by the parser, so we'll convert those to NaN's by using the `errors='coerce'` argument.

In [ ]:
df['uncertainty [nm]'] = pd.to_numeric(df['uncertainty [nm]'], errors='coerce')

Finally, we need to replace any Inf's with NaN's and then drop the NaN's. We'll reindex the final result.

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna().describe()

In [ ]:
df.reindex()
print('Done.')

## Normal localization processing
Now that the data is cleaned up a bit, we'll proceed with our normal processing.

In [ ]:
df.describe()

In [ ]:
FilterLLR  = ds.Filter('loglikelihood', '<', 400)
FilterSig1 = ds.Filter('sigma [nm]',    '>', 100)
FilterSig2 = ds.Filter('sigma [nm]',    '<', 180)
df2        = FilterSig2(FilterSig1(FilterLLR(df)))

In [ ]:
df2.describe()

## Display the 2D histogram to visually identify fiducials

Now we need to make a 2D histogram to see whether the fiducial localizations are apparent.

In [ ]:
import importlib
importlib.reload(ds)

In [ ]:
# Find maximum x or y coordinate
maxPos    = np.max([df2['x [nm]'].max(), df2['y [nm]'].max()])
pixelSize = 100 # nm

numBins = int(maxPos / pixelSize)
plt.hist2d(df2['x [nm]'], df2['y [nm]'], bins = numBins)
plt.show()

In [ ]:
plt.close()

In [ ]:
corrector = ds.FiducialDriftCorrect(mergeRadius           = 50,
                                    offTime               = 1,
                                    minSegmentLength      = 20,
                                    minFracFiducialLength = 0.4,
                                    neighborRadius        = 500,
                                    smoothingWindowSize   = 625,
                                    smoothingFilterSize   = 500,
                                    searchRegions         = [{'xMin' :  2200,
                                                              'xMax' :  2800,
                                                              'yMin' : 33200,
                                                              'yMax' : 33700}])

In [ ]:
df3 = corrector(df2)

In [ ]:
# Find maximum x or y coordinate
maxPos    = np.max([df3['x [nm]'].max(), df3['y [nm]'].max()])
pixelSize = 100 # nm

numBins = int(maxPos / pixelSize)
plt.hist2d(df3['x [nm]'], df3['y [nm]'], bins = numBins)
plt.show()

In [ ]:
x0 = corrector.fiducialTrajectories[0]['x'].iloc[[0]].as_matrix()
plt.plot(corrector.fiducialTrajectories[0]['frame'], corrector.fiducialTrajectories[0]['x'] - x0)
plt.plot(corrector.avgSpline.index,corrector.avgSpline['xS'])
plt.plot(np.arange(100,50000), corrector.splines['xS'][0](np.arange(100,50000)) - x0)
plt.show()

In [ ]:
# BE SURE TO DROP NULLS FIRST
df3.dropna(inplace = True)
df3.to_csv('fullData.csv', index = False)

In [ ]:
df5 = df2[df2['frame'] > 4000]

In [ ]:
df5.to_csv('partialData.csv', index = False)

In [ ]:
df2.dropna(inplace = True)